In [ ]:
!who

In [3]:
import logging
import os.path

import numpy as np
import pymc as pm
import aesara.tensor as at
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas as pd
import warnings
import extract_correct_csv

from deepemogp import feature_extractor
from deepemogp.signal import physio as physio
from deepemogp import datasets as datasets
from deepemogp.signal import behavior as behavior


import logging
import pymc as pm
import numpy as np
import arviz as az

import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

2023-01-13 16:19:35.910023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-13 16:19:36.064704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-13 16:19:36.064730: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-13 16:19:36.769357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
import jax
jax.default_backend()

In [ ]:
jax.devices()

In [4]:
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

scaler = StandardScaler()

valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_k_list = list(range(1, 10))

k=5
i =2
num_trials_to_remove = 48

string_subject = extract_correct_csv.read_correct_subject_csv(i)
csv_ = 'data/LookAtMe_0' + string_subject + '.csv'
# csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_045.csv'
global_data = pd.read_csv(csv_, sep='\t')
y = np.array(list([int(d > 2) for d in global_data['rating']]))
e_labels = y[:, np.newaxis]  # rating > 2
e_labels = e_labels[num_trials_to_remove:]
N_e = e_labels.shape[0]
D_e = e_labels.shape[1]

TRIAL = 160

hr = pd.read_csv('data/features/hr/'+str(i)+'.csv')
hr = hr[num_trials_to_remove:]

eda = pd.read_csv('data/features/eda/' + str(i) + '.csv')
eda = eda[num_trials_to_remove:]

pupil = pd.read_csv('data/features/pupil/' + str(i) + '.csv')
pupil = pupil[num_trials_to_remove:]

N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]
K = k

In [5]:
print(N_eda,D_eda)

112 60


In [8]:
print(pm.__version__)
print(tf.__version__)
print(tfp.__version__)

4.2.2
2.11.0
0.19.0


In [10]:
@pm.model
def model():
    # dati osservabili
    hr_data = yield pm.MutableData("hr_data", hr.T)
    pupil_data = yield pm.MutableData("pupil_data", pupil.T)
    eda_data = yield pm.MutableData("eda_data", eda.T)

    e_data = yield pm.ConstantData("e_data", e_labels.T)

    # matrici pesi
    Whr = yield pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    Wpupil = yield pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]),
                       shape=[D_pupil, K])

    Weda = yield pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    # weight matrix for pain expectation.
    # check mu,sigma,shape
    We = yield pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e, K]), shape=[D_e, K])

    # latent space
    c = yield pm.Normal('c', mu=at.zeros([N_hr, K]), sigma=at.ones([N_hr, K]), shape=[N_hr, K])

    # dati dell'hrv interpretati come una gaussiana
    mu_hr = yield pm.Normal('mu_hr', Whr.dot(c.T), at.ones([D_hr, N_hr]))  # hyperprior 1
    sigma_hr = yield pm.Exponential('sigma_hr', at.ones([D_hr, N_hr]))  # hyperprior 2
    x_hr = yield pm.Normal('x_hr', mu=mu_hr, sigma=sigma_hr, shape=[D_hr, N_hr], observed=hr_data)

    # dati della dilatazione pupille interpretati come una gaussiana
    mu_pupil = yield pm.Normal('mu_pupil', Wpupil.dot(c.T), at.ones([D_pupil, N_pupil]))  # hyperprior 1
    sigma_pupil = yield pm.Exponential('sigma_pupil', at.ones([D_pupil, N_pupil]))  # hyperprior 2
    x_pupil = yield pm.Normal('x_pupil', mu=mu_pupil, sigma=sigma_pupil, shape=[D_pupil, N_pupil],
                        observed=pupil_data)

    # eda
    mu_eda = yield pm.Normal('mu_eda', Weda.dot(c.T), at.ones([D_eda, N_eda]))  # hyperprior 1
    sigma_eda = yield pm.Exponential('sigma_eda', at.ones([D_eda, N_eda]))  # hyperprior 2
    x_eda = yield pm.Normal('x_eda', mu=mu_eda, sigma=sigma_eda, shape=[D_eda, N_eda], observed=eda_data)

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    x_e = yield pm.Bernoulli('x_e', p=pm.math.sigmoid(We.dot(c.T)), shape=[D_e, N_e], observed=e_data)
    return x_e

TypeError: 'module' object is not callable

In [ ]:

with sPPCA:
    approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    trace = approx.sample(500)

In [ ]:
# az.plot_trace(trace);
with sPPCA:
    # update values of predictors:
    # pm.set_data({"pupil_data": pupil, "hr_data": hr, "eda_data": eda})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
eda_pred = posterior_predictive.posterior_predictive["x_eda"]

In [ ]:
eda_pred = np.squeeze(eda_pred.mean('draw', keepdims='false')[0]).to_numpy()

In [ ]:
edapred_ = eda_pred.T

eda_ = eda.to_numpy()

corrlist = []
for i in range(112):
    res = np.corrcoef(eda_[i], edapred_[i])[0][1]
    corrlist.append(res)
    print('trial '+str(i)+ ' corr: '+str(res.round(3)))

In [ ]:
def ccc(x, y):
    ''' Concordance Correlation Coefficient'''
    sxy = np.sum((x - x.mean()) * (y - y.mean())) / x.shape[0]
    rhoc = 2 * sxy / (np.var(x) + np.var(y) + (x.mean() - y.mean()) ** 2)
    return rhoc

In [ ]:
ccc(edapred_[0],eda_[0])

In [ ]:
eda_[0]

In [ ]:
edapred_[0].shape

In [ ]:
x=eda_[0]
y=eda_pred[0]

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
x- x.mean()

In [ ]:
y-y.mean()

In [ ]:
(x - x.mean()) * (y - y.mean())


In [ ]:
sxy= np.sum() / x.shape[0]

In [ ]:
conc = ccc(eda_[0], eda_pred[0])

In [ ]:
pearson_list = []
concord_list = []
for i in range(112):
    pear = np.corrcoef(eda_[i], edapred_[i])[0][1]
    conc = ccc(eda_[i], eda_pred[i])
    pearson_list.append(pear)
    concord_list.append(conc)
    # print('trial ' + str(i) + ' corr: ' + str(res.round(3)))

In [ ]:
mean_pear = round(np.mean(pearson_list), 4)
mean_corc = round(np.mean(concord_list), 4)

In [ ]:
mean_subj = round(np.mean(corrlist),4)

In [ ]:

        logging.basicConfig(level=logging.INFO, filename="logfile", filemode="a+",
                            format="%(asctime)-15s %(levelname)-8s %(message)s")
        logging.info("Mean corr coeff eda-hr using subj: " + str(subj_) + " " + str(round(mean_subj, 2)) + " script: " +
             os.path.basename(__file__) + "latent space dims: " + str(K))
